In [2]:
import re
import json
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
filepath = 'exxon_climate_change_2021-01-01_2022-01-01.txt'

### Create a dataframe for the first dictionary (failed right now)

Take a look at our dirst dictionary

In [5]:
df_0_json = pd.read_json(filepath, lines=True, nrows=1)

In [6]:
df_0_json['lang']

0    und
Name: lang, dtype: object

In [4]:
df_0_json

,lang,id,possibly_sensitive,author_id,text,referenced_tweets,created_at,source,entities,in_reply_to_user_id,reply_settings,conversation_id,public_metrics,author,in_reply_to_user,__twarc
0,und,1477065589361315840,False,1460383957262880768,@tjbabel @Catherine_LCP @SamBoik @Cernovich ht...,"[{'type': 'replied_to', 'id': '147706468563978...",2021-12-31 23:54:13+00:00,Twitter for iPhone,"{'urls': [{'start': 44, 'end': 67, 'url': 'htt...",1733391854,everyone,1476711967607853056,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'verified': False, 'created_at': '2021-11-15T...","{'verified': False, 'created_at': '2013-09-05T...",{'url': 'https://api.twitter.com/2/tweets/sear...


 ### Try to Create a dataframe by every columns

In [7]:
#load all the JSON objects in one list as dictionaries
data = []
with open(filepath,'r') as f:
    for line in f:
        data.append(json.loads(line))

In [84]:
data[0]['referenced_tweets']


[{'type': 'replied_to',
  'id': '1477064685639782404',
  'lang': 'en',
  'possibly_sensitive': False,
  'author_id': '1733391854',
  'text': '@owen_lavine @Catherine_LCP @SamBoik @Cernovich Ever heard of the carbon cycle? Google it.',
  'entities': {'annotations': [{'start': 80,
     'end': 85,
     'probability': 0.4095,
     'type': 'Organization',
     'normalized_text': 'Google'}],
   'mentions': [{'start': 0,
     'end': 12,
     'username': 'owen_lavine',
     'id': '1460383957262880776',
     'verified': False,
     'created_at': '2021-11-15T23:07:29.000Z',
     'public_metrics': {'followers_count': 12,
      'following_count': 245,
      'tweet_count': 963,
      'listed_count': 0},
     'name': 'Owen Lavine',
     'description': 'Journalist - Cal Poly SLO ‘24 - LA',
     'protected': False,
     'url': '',
     'profile_image_url': 'https://pbs.twimg.com/profile_images/1460384644122099713/ToEk8Ndb_normal.jpg',
     'pinned_tweet_id': '1482061096068341760'},
    {'start': 13,
 

In [54]:
data[5467]['text']

'RT @ChristineArena: 7/ From 2008 to 2018, Edelman made nearly $500,000,000 in contracts with trade associations opposed to climate action.…'

In [65]:
data[0]['text']

'@tjbabel @Catherine_LCP @SamBoik @Cernovich https://t.co/a5p9XOE7b9'

In [66]:
data[0]['entities']['urls']

[{'start': 44,
  'end': 67,
  'url': 'https://t.co/a5p9XOE7b9',
  'expanded_url': 'https://unearthed.greenpeace.org/2021/06/30/exxon-climate-change-undercover/',
  'display_url': 'unearthed.greenpeace.org/2021/06/30/exx…'}]

In [22]:
data[99]['possibly_sensitive']

True

In [20]:
#look at the first dictionary 
data[0]

{'lang': 'und',
 'id': '1477065589361315842',
 'possibly_sensitive': False,
 'author_id': '1460383957262880776',
 'text': '@tjbabel @Catherine_LCP @SamBoik @Cernovich https://t.co/a5p9XOE7b9',
 'referenced_tweets': [{'type': 'replied_to',
   'id': '1477064685639782404',
   'lang': 'en',
   'possibly_sensitive': False,
   'author_id': '1733391854',
   'text': '@owen_lavine @Catherine_LCP @SamBoik @Cernovich Ever heard of the carbon cycle? Google it.',
   'entities': {'annotations': [{'start': 80,
      'end': 85,
      'probability': 0.4095,
      'type': 'Organization',
      'normalized_text': 'Google'}],
    'mentions': [{'start': 0,
      'end': 12,
      'username': 'owen_lavine',
      'id': '1460383957262880776',
      'verified': False,
      'created_at': '2021-11-15T23:07:29.000Z',
      'public_metrics': {'followers_count': 12,
       'following_count': 245,
       'tweet_count': 963,
       'listed_count': 0},
      'name': 'Owen Lavine',
      'description': 'Journalist - C

 #### Put the text data in one list

In [7]:
lst_text = []
for i in data:
    lst_text.append(i['text'])

 #### Put the author info in list

In [8]:
data[0].keys()

dict_keys(['lang', 'id', 'possibly_sensitive', 'author_id', 'text', 'referenced_tweets', 'created_at', 'source', 'entities', 'in_reply_to_user_id', 'reply_settings', 'conversation_id', 'public_metrics', 'author', 'in_reply_to_user', '__twarc'])

 ##### 'id', 'author_id', 'conversation_id', 'author'

In [9]:
#create lists for id, author id and conversation
lst_id = []
lst_author_id = []
lst_conversation_id = []
for i in data:
    lst_id.append(i['id'])
    lst_author_id.append(i['author_id'])
    lst_conversation_id.append(i['conversation_id'])

There are too much info in author dictionary

In [10]:
data[0]['author']

{'verified': False,
 'created_at': '2021-11-15T23:07:29.000Z',
 'public_metrics': {'followers_count': 12,
  'following_count': 245,
  'tweet_count': 963,
  'listed_count': 0},
 'name': 'Owen Lavine',
 'description': 'Journalist - Cal Poly SLO ‘24 - LA',
 'protected': False,
 'username': 'owen_lavine',
 'url': '',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1460384644122099713/ToEk8Ndb_normal.jpg',
 'id': '1460383957262880776',
 'pinned_tweet_id': '1482061096068341760'}

In [11]:
#lst verified or not, author name and author username
lst_author_verified = []
lst_author_name = []
lst_author_username = []
for i in data:
    lst_author_verified.append(i['author']['verified'])
    lst_author_name.append(i['author']['name'])
    lst_author_username.append(i['author']['username'])

 ##### Make creation time

In [12]:
#list creation time
lst_twt_creation_time = []
for i in data:
    lst_twt_creation_time.append(i['created_at'])

##### Put what we have in one dataframe

lst_text,lst_id, lst_author_id,lst_conversation_id,lst_author_verified, lst_author_name, lst_author_username,lst_twt_creation_time

In [13]:
df = pd.DataFrame({'text': lst_text,
                  'id': lst_id,
                  'author_id': lst_author_id,
                  'conversation_id': lst_conversation_id,
                  'author_verified': lst_author_verified,
                  'author_name': lst_author_name,
                  'author_username': lst_author_username,
                  'twt_creation_time': lst_twt_creation_time
                  })

In [14]:
df.head()

,text,id,author_id,conversation_id,author_verified,author_name,author_username,twt_creation_time
0,@tjbabel @Catherine_LCP @SamBoik @Cernovich ht...,1477065589361315842,1460383957262880776,1476711967607853064,False,Owen Lavine,owen_lavine,2021-12-31T23:54:13.000Z
1,@tjbabel @Catherine_LCP @SamBoik @Cernovich ht...,1477064791499759617,1460383957262880776,1476711967607853064,False,Owen Lavine,owen_lavine,2021-12-31T23:51:03.000Z
2,RT @maxberger: I'm thinking about writing a sa...,1477059914015125507,2732778706,1477059914015125507,False,Tai Chi,TaiWendysb,2021-12-31T23:31:40.000Z
3,"RT @zia_kat: @Mum2Mrs @oshima9 no, capitalists...",1477056800889323523,414239235,1477056800889323523,False,Richard Estes,oshima9,2021-12-31T23:19:18.000Z
4,RT @Carpe_DMT: @ScottGrimnes @MaMan564 @SoFarF...,1477046502400868359,824946672673361920,1477046502400868359,False,Kris,writing_callous,2021-12-31T22:38:23.000Z


In [16]:
df[0:5]['text']

0    @tjbabel @Catherine_LCP @SamBoik @Cernovich ht...
1    @tjbabel @Catherine_LCP @SamBoik @Cernovich ht...
2    RT @maxberger: I'm thinking about writing a sa...
3    RT @zia_kat: @Mum2Mrs @oshima9 no, capitalists...
4    RT @Carpe_DMT: @ScottGrimnes @MaMan564 @SoFarF...
Name: text, dtype: object